In [1]:
import os
from numpy import array

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

## GPU 설정

In [2]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## 데이터

In [3]:
docs = ["너무 재밌네요", 
        "최고에요", 
        "참 잘 만든 영화에요", 
        "추천하고 싶은 영화입니다", 
        "한번 더 보고싶네요",
        "글쎄요",
        "별로에요",
        "생각보다 지루하네요",
        "연기가 어색해요",
        "재미없어요"]

In [4]:
classes = array([1,1,1,1,1,0,0,0,0,0])

## tokenizer

In [5]:
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화에요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로에요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [6]:
x = token.texts_to_sequences(docs)
print("리뷰 텍스트, 토근화 결과: \n", x)

리뷰 텍스트, 토근화 결과: 
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


## padding
길이가 다른 데이터를 4로 맞추기 

In [7]:
padded_x = pad_sequences(x, 4)
print("패딩 결과: \n", padded_x)

패딩 결과: 
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


## Model

In [8]:
word_size = len(token.word_index)+1
word_size

21

In [9]:
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 8)              168       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


## Train

In [10]:
model.compile(optimizer="adam", loss = 'binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)

Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 0.6995 - accuracy: 0.4000
Epoch 2/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6977 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6958 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6939 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6921 - accuracy: 0.5000
Epoch 6/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6903 - accuracy: 0.6000
Epoch 7/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6884 - accuracy: 0.6000
Epoch 8/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6866 - accuracy: 0.7000
Epoch 9/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6847 - accuracy: 0.7000
Epoch 10/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6829 - accuracy: 0.7000
Epoch 11/20
1/1 [=======================

## Test

In [11]:
print("Accuracy: ", model.evaluate(padded_x, classes)[1])

1/1 [==============================] - 0s 107ms/step - loss: 0.6619 - accuracy: 0.9000
Accuracy:  0.8999999761581421
